This notebook is a narrative exploration of efforts to predict the number of citations per year a paper will receive,
based on data available at time of publication.

A full exploration of 25 different prediction models can be found [here](askfhs)

In this notebook I look to predict the number of citations a paper will received based upon
* the words used in the abstract
* physics-inspired semantic metrics (implemented in github.com/zhafen/cc and employed in Imel & Hafen in prep)
* metadata (publishing journal, number of authors, number of pages, etc.)

The raw code for this analysis can be [found here](https://github.com/zhafen/work-sample).

I aimed to keep this work sample clean, so please reach out if you have questions about details I have not included.

You can jump to punchlines in the analysis via these links:

# User-Defined Parameters

This dictionary contains various user-defined parameters. They will be explained when they are utilized.

In [ ]:
pm = dict(
    
    # Data selection
    data_dir = '/Users/zhafen/data/literature_topography',
    region_number = 8,
    convergence_degree = 3,
    kernel_size = 16,
    
    # Features
    numerical_variables = [
        'age',
        'references_count',
        'page_count',
        'log_author_count',
    ],
    categorical_variables = [
        'journal_filtered',
    ],
    semantic_variables = [
        'density',
        'fringe_factor',
    ],
    
    # Modeling
    scoring = 'neg_root_mean_squared_error',
    
)

# Data

I use publication abstracts and metadata pulled from the [NASA astrophysics data sytem](https://ui.adsabs.harvard.edu) via [the official API](https://ui.adsabs.harvard.edu/help/api/). The analyzed publications are from a randomly-chosen physics or astrophysics specialization.

I externally preprocessed the abstract data with natural language processing (including tokenizing, stemming, and removing filler words), and each abstract has a corresponding bag-of-words representation.

## Load raw data

In [ ]:
import numpy as np
import os

In [ ]:
# My custom non-relational-data-management package
import verdict
# My library for NLP analysis of scientific abstracts
from cc import atlas, cartography, utils

In [ ]:
# Load summary information.
# I analyzed several randomly chosen specializations ("regions"), of which we are choosing an arbitrary one.
summary_data_fp = os.path.join( pm['data_dir'], 'regions', 'regions_summary.h5' )
data = verdict.Dict.from_hdf5( summary_data_fp )
data_k = data['regions'][str(pm['region_number'])]

In [ ]:
# Class for management of abstracts
atlas_dir = os.path.join( pm['data_dir'], 'regions', 'region_{}'.format( pm['region_number'] ) )
a = atlas.Atlas( atlas_dir, load_bibtex=False )

In [ ]:
# Class for analysis of vectorized abstracts
projection = a.vectorize(
    verbose = True,
)
c = cartography.Cartographer( **projection )

In [ ]:
# Retrieve metrics I calculated in external pre-processing
metrics_fp = os.path.join( atlas_dir, 'topography_metrics.h5' )
metrics = verdict.Dict.from_hdf5( metrics_fp )

In [ ]:
# Not all the publications are viable for analysis.
# I've saved information about what publications are viable, and here we load the identifying information.
converged_kernel_size = data_k['converged_kernel_size'][:,-pm['convergence_degree']]
converged = converged_kernel_size >= pm['kernel_size']
publications = c.publications[converged]
inds = np.arange( c.publications.size )[converged]

In [ ]:
# Select word vectors
v = c.vectors[inds]

## Format into a DataFrame

In [ ]:
import copy
import pandas as pd
import warnings

In [ ]:
# Make into a dataframe, for convenience.
df_data = copy.deepcopy( metrics )
df_data['projection_ind'] = inds
df = pd.DataFrame(
    data = df_data._storage,
    index=publications,
)

In [ ]:
# Drop publications with no citations.
# This catches all grant submissions, etc.
df = df.loc[np.invert( np.isclose( df['citations_per_year'], 0. ) )]

In [ ]:
# Add logscale versions for some variables
for column in [ 'density', 'citations_per_year', ]:
    df['log_{}'.format( column )] = np.log10( df[column] )

In [ ]:
# Drop columns for which we're missing abstract data (will show up as a nan in density)
df.dropna(subset=['density',], inplace=True)

## Derive or retrieve additional quantities

In [ ]:
import tqdm

### ADS metadata

In [ ]:
citation_keys = []
additional_data = {
    'references_count': [],
    'pages': [],
    'author': [],
    'journal': [],
    'title': [],
    'abstract_character_count': [],
    'entry_date': [],
}
for citation_key, p in tqdm.tqdm( a.data.items() ):
    
    citation_keys.append( citation_key )
    
    # number of references
    if p.references is None:
        additional_data['references_count'].append( 0 )
    else:
        additional_data['references_count'].append( len( p.references ) )
    
    # Citation info
    for key in [ 'pages', 'author', 'journal', 'title' ]:
        try:
            additional_data[key].append( p.citation[key] )
        except KeyError:
            additional_data[key].append( pd.NA )
            
    # Abstract
    additional_data['abstract_character_count'].append( len( p.abstract_str() ) )
    
    # Entry date
    additional_data['entry_date'].append( p.entry_date )

In [ ]:
# Convert to datetime
additional_data['entry_date'] = pd.to_datetime( additional_data['entry_date'] )

In [ ]:
# Join it onto the existing dataframe
additional_df = pd.DataFrame( data=additional_data, index=citation_keys )
df = df.join( additional_df )

### Index

In [ ]:
df['ind'] = np.arange( df.index.size )

### Page count

In [ ]:
# Setup data structurs
df['page_count'] = np.full( len( df ), np.nan )

In [ ]:
# Get rid of the "L" in front of the pages for publications submitted to letters.
pages_str = df['pages'].str.replace( 'L|P', '' )

In [ ]:
# Split into two to take the difference
pages_split = pages_str.str.split( '-', expand=True )

In [ ]:
# Identify the parseable data
is_not_none = np.invert( pages_split[1].isnull() )
is_numeric = pages_split[1].str.isnumeric()
is_page_range = is_not_none & is_numeric

In [ ]:
# For the valid page ranges, set the page count
df.loc[is_page_range,'page_count'] = (
    pages_split[1].loc[is_page_range].astype( int )
    - pages_split[0].loc[is_page_range].astype( int )
)

In [ ]:
# There can be one or two edge cases where there's a negative page count because of the formatting
df.loc[df['page_count']<0,'page_count'] = np.nan

### Author count

In [ ]:
df['author_count'] = df['author'].str.split( ' and ' ).apply( len )
df['log_author_count'] = df['author_count'].apply( np.log10 )

### Title character count

In [ ]:
df['title_character_count'] = df['title'].str.len()

### Journals, filtered

In [ ]:
# Find the most common journals
df_grouped = df.groupby( 'journal' )
journal_entry_count = df_grouped.size().sort_values( ascending=False )
most_common_journals = journal_entry_count.iloc[:5].index

In [ ]:
most_common_journals

In [ ]:
# Make a new column accordingly
df['journal_filtered'] = df['journal'].copy()
is_not_common_journal = np.invert( df['journal'].isin( most_common_journals ) )
df.loc[is_not_common_journal,'journal_filtered'] = 'other'
df.loc[df['journal'].isna(),'journal_filtered'] = 'other'

### Word vectors

In [ ]:
v = c.vectors[df['projection_ind']]

## Summarize data

There are two main data containers:
`df`, which contains all the metadata and derived quantities,
and `v`, which contains the word vectors.

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
v.shape

## Split testing and training data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split dataframe
df_train, df_test = train_test_split( df, test_size=0.2, random_state=42 )

In [ ]:
# Split word vector input
v_train = v[df_train['ind']]
v_test = v[df_test['ind']]

In [ ]:
# Split semantic input
M_train = df_train[pm['semantic_variables']].values
M_test = df_test[pm['semantic_variables']].values

In [ ]:
# Split metadata input
X_train_df = df_train[pm['numerical_variables'] + pm['categorical_variables']]
X_test_df = df_test[pm['numerical_variables'] + pm['categorical_variables']]

In [ ]:
# Split output
y_train = df_train['log_citations_per_year'].values
y_test = df_test['log_citations_per_year'].values

# Exploratory Data Analysis

I've explored this dataset thoroughly elsewhere, so here I'll just visually summarize the dataset.

In [ ]:
import seaborn as sns

sns.set_style( 'whitegrid' )

## Citations per year

In [ ]:
sns.histplot(
    df_train,
    x = 'log_citations_per_year',
)

## Numerical variables

In [ ]:
pairplot = sns.pairplot(
    df_train,
    x_vars = pm['numerical_variables'],
    y_vars = [ 'log_citations_per_year',],
    kind = 'hist',
    # plot_kws = { 'line_kws': { 'color': 'k', }, },
)
pairplot.axes[0,1].set_xlim( 0, 210 )
pairplot.axes[0,2].set_xlim( 0, 40 )

## Categorical variable

In [ ]:
sns.violinplot(
    df_train,
    x = 'journal_filtered',
    y = 'log_citations_per_year',
)

# Modeling

## Preprocessing

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, KFold

In [ ]:
# Two things for our numerical variables:
# Imputation of missing values and scaling by mean and std
numerical_preprocessing = Pipeline(
    [
        ( 'impute', SimpleImputer( strategy='mean' ) ),
        ( 'scale', StandardScaler() ),
    ]
)

In [ ]:
# Preprocessing for word vectors is just scaling
vector_preprocessing = Pipeline(
    [
        ( 'scale', Normalizer() ),
    ]
)

In [ ]:
# There is a subset of the numerical variables that I refer to as "semantic".
# These variables contain metrics that measure the relationship of the words in a publication to words in other publications.
semantic_preprocessing = Pipeline(
    [
        ( 'scale', StandardScaler() ),
    ]
)

In [ ]:
# Combine the numerical preprocessing with onehot encoding for the categorical variable
metadata_preprocessing = ColumnTransformer( [
        ( 'numerical', numerical_preprocessing, pm['numerical_variables'] ),
        ( 'onehot', OneHotEncoder(), pm['categorical_variables'] ),
] )

In [ ]:
# Set up a kfold object for cross validation
kfold = KFold(
    n_splits = 5,
    shuffle = True,
    random_state = 1532
)

## A Baseline Model

We use the mean log citations per year as the baseline.

In [ ]:
from sklearn.base import BaseEstimator

In [ ]:
class Baseline( BaseEstimator ):
    '''The baseline model is just the mean. We put it into a class
    for full consistency with all future models.'''
    
    def fit( self, X , y):
        
        self.estimate = y.mean()
        
    def predict( self, X ):
        
        return np.full( X.shape[0], self.estimate )

model = Baseline()

In [ ]:
# Object for storing data
crossvals = {}

In [ ]:
# Perform and store cross validation
crossvals['baseline'] = verdict.Dict( cross_validate(
    estimator = model,
    X = X_train,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

Because the baseline is the mean we expect the RMSE for the individual folds to be similar to the standard deviation of the full sample.

In [ ]:
-crossvals['baseline']['test_score']

In [ ]:
sample_mean = y_train.mean()
sample_std = y_train.std()
sample_std

## A Naive Model: Simple Linear Regression

For our first model we'll see if we can just use linear least squares regression with the word vectors as input.
This is a pretty silly model: we're limited to linear order because of the high dimensionality of the word vectors, and it is unlikely that a single line can describe the citation relationship.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
model = Pipeline(
    [
        ( 'preprocessing', vector_preprocessing ),
        ( 'poly', PolynomialFeatures( degree=1 ) ),
        ( 'reg', LinearRegression( fit_intercept=False ) ),
    ]
)

In [ ]:
# Perform and store cross validation
crossvals['linear_regression'] = verdict.Dict( cross_validate(
    estimator = model,
    X = v_train,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def rmse_swarmplot( crossval, y_lim=(sample_std-0.15, sample_std+0.15) ):    
    
    # Format data
    df_data = -1. * verdict.Dict( crossvals ).inner_item( 'test_score' )
    crossval_df = pd.DataFrame( df_data._storage ).melt( var_name='model', value_name='rmse' )
    
    # Visualize
    fig = plt.figure( figsize=(len(df_data)*1.5, 3) )
    ax = plt.gca()
    
    # Plot itself
    sns.swarmplot(
        data = crossval_df,
        x = 'model',
        y = 'rmse',
        ax = ax,
    )
    
    # Mark the analytic baseline value for comparison
    ax.axhline(
        sample_std,
        color = '0.5',
        linestyle = '--',
        linewidth = 0.75,
    )

    ax.set_xlabel( 'model' )
    ax.set_ylabel( r'RMSE in log_citations_per_year' )
    
    ax.set_ylim( y_lim )
    
    return fig

In [ ]:
fig = rmse_swarmplot( crossvals, y_lim=(sample_std-0.5, sample_std+0.5) )

No surprise, this model doesn't perform particularly well.

## A Basic Phenomenological Model: Random Forest

For a more-sophisticated phenomenological description of the data, let's try a random forest.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = Pipeline(
    [
        ( 'preprocessing', vector_preprocessing ),
        ( 'reg', RandomForestRegressor( max_depth=3, n_estimators=200 ) ),    ]
)

In [ ]:
# Perform and store cross validation
crossvals['random_forest'] = verdict.Dict( cross_validate(
    estimator = model,
    X = v_train,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

In [ ]:
fig = rmse_swarmplot( crossvals )

Better performance than the baseline! We're getting somewhere.

## A Better Phenomoenological Model: Gradient Boosting

If the random forest model went well, how about a gradient boosting model with decision trees as the base?

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
model = Pipeline(
    [
        ( 'preprocessing', vector_preprocessing ),
        ( 'reg', GradientBoostingRegressor() ),
    ]
)

In [ ]:
# Perform and store cross validation
crossvals['grad_boost'] = verdict.Dict( cross_validate(
    estimator = model,
    X = v_train,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

In [ ]:
# Let's go ahead and drop the linear regression model so it's not distracting us.
del crossvals['linear_regression']

In [ ]:
fig = rmse_swarmplot( crossvals, y_lim=(sample_std-0.2, sample_std+0.2) )

Great. We have a model that does noticeably better than the baseline.

In [ ]:
print( 'We can predict the number of citations to within a factor of ~{:.2g} on average.'.format( 10.**-crossvals['grad_boost']['test_score'].mean() ) )

There is still a significant amount of error in the prediction: a factor of a few in predicting the number of citations per year. This is perhaps to be expected: it would be surprising if the main thing driving citation trends could be predicted simply via the words used, with no information about how those words relate to one another (e.g. via an N-gram language model).

## An Informed Model: K Nearest Neighbors

So far we've been treating the word vectors as generic data. To further improve our model, and add some explanatory power, let's make use of our knowledge about what the vectors _are_: the language used in a scientific abstract.

Here's a simple hypothesis:
The number of citations a paper receives correlates with the number of citations papers on similar topics (i.e. using similar words) receive. Fortunately this is a simple, well-defined model: the K Nearest Neighbors model.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
model = Pipeline(
    [
        ( 'preprocessing', vector_preprocessing ),
        ( 'knn', KNeighborsRegressor( n_neighbors=32 ) ),
    ]
)

In [ ]:
# Perform a parameter search for the number of neighbors to use
param_grid = {
    'knn__n_neighbors': [ 4, 16, 32, 64, 128, 256, ],
}
search = GridSearchCV( model, param_grid, )
search.fit( v_train, df_train['log_citations_per_year'] )
model = search.best_estimator_
search.best_params_

In [ ]:
# Perform and store cross validation
crossvals['KNN'] = verdict.Dict( cross_validate(
    estimator = model,
    X = v_train,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

In [ ]:
fig = rmse_swarmplot( crossvals )

KNN does comparably well to gradient boosting, but is much more interpretable!

## Another Informed Model: Density and Asymmetry

Related to K Nearest Neighbors, we can ask if the local geometry of the hyperspace is related to citations received. In particular, we'll define two metrics:

density, which tracks the number of similar papers,
$${\rm density} = \frac{K}{{\rm distance\,to\,the\,farthest\,neighbor}}$$

and "fringe factor", which tracks if a paper uses language in a new direction relative to existing similar papers.
$${\rm fringe\,factor} = \frac1K \sum_{i=1}^{K} \frac{\vec v - \vec v_i}{|\vec v - \vec v_i|} $$
The concept of edginess comes from the concept of force balance.

I calculated these quantities in preprocessing.

In [ ]:
sns.histplot(
    df_train,
    x = 'density',
)

In [ ]:
sns.histplot(
    df_train,
    x = 'fringe_factor',
)

Most of the meaningful modeling is in the calculation of these two quantities, so we'll fit a simple linear regression.

In [ ]:
semantic_model = Pipeline(
    [
        ( 'preprocessing', semantic_preprocessing ),
        ( 'poly', PolynomialFeatures( degree=1 ) ),
        ( 'reg', LinearRegression( fit_intercept=False ) ),
    ]
)

In [ ]:
# Perform and store cross validation
crossvals['density_and_fringe_factor'] = verdict.Dict( cross_validate(
    estimator = semantic_model,
    X = M_train,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

In [ ]:
fig = rmse_swarmplot( crossvals )

Regressing on the explanatory variables of density and fringe factor does not give us the best model, but nevertheless holds some predictive power.

## A Model Utilizing Metadata

Our analysis so far has been limited to the word content of the abstract. But presumably the number of citations received is a function of much more than the words employed. I have gathered a number of additional attributes for each publication, and we will regress onto those too.

In [ ]:
X_train_df

I will regress with gradient boosting, but in exploratory modeling I discoverd that most other reasonable ML models perform similarly.

In [ ]:
metadata_model = Pipeline(
    [
        ( 'preprocessing', metadata_preprocessing ),
        ( 'reg', GradientBoostingRegressor() ),
    ]
)

In [ ]:
# Perform and store cross validation
crossvals['metadata'] = verdict.Dict( cross_validate(
    estimator = metadata_model,
    X = X_train_df,
    y = y_train,
    cv = kfold,
    scoring = 'neg_root_mean_squared_error',
    return_estimator = True,
) )

In [ ]:
fig = rmse_swarmplot( crossvals )

The metadata contains a lot of information useful for predicting citations!

## A Neural Net

We'll wrap up our individual models with a simple neural net regressed onto the word vectors.

In [ ]:
import keras

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Simple multilayer model
neural_net = keras.models.Sequential()
neural_net.add( keras.layers.Input( shape=(v_train.shape[1],) ) )
neural_net.add( keras.layers.Dense( 16, activation='relu', ) )
neural_net.add( keras.layers.Dense( 16, activation='relu' ) )
neural_net.add( keras.layers.Dense( 1, ) )
neural_net.compile( loss='mean_squared_error' )

In [ ]:
# Actually build the model
model = Pipeline(
    [
        ( 'preprocessing', vector_preprocessing ),
        ( 'reg', neural_net ),
    ]
)

In [ ]:
# Perform KFold cross validation
# We could use the exact same code as before, but this allows a little additional control.
crossvals['neural_net'] = {'test_score':[]}
for train_index, test_index in kfold.split(v_train):
        
    # Fit and predict
    model.fit( v_train[train_index].toarray(), y_train[train_index] )
    log_cpy_pred = model.predict( v_train[test_index].toarray() )
    
    # Compare prediction
    rmse = -np.sqrt( mean_squared_error( y_train[test_index], log_cpy_pred ) )
    crossvals['neural_net']['test_score'].append( rmse )
crossvals['neural_net']['test_score'] = np.array( crossvals['neural_net']['test_score'] )

In [ ]:
fig = rmse_swarmplot( crossvals, y_lim=(sample_std-0.5, sample_std+0.5 ) )

While arguably the biggest blackbox of the considered models, the simple neural net performs remarkably well for predicting citations given the words used, at least on the training set.

## Putting it All Together: Voting Regression

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import VotingRegressor

In [ ]:
# Combine word vectors, semantic variables, and metadata into a single feature array
X_train = metadata_preprocessing.fit_transform( X_train_df )
n_sample = X_train.shape[0]
n_X = X_train.shape[1]
n_M = M_train.shape[1]
n_v = v_train.shape[1]
A_train = np.zeros( shape=(n_sample, n_X + n_M + n_v) )
A_train[:,:n_X] = X_train
A_train[:,n_X:n_X+n_M] = M_train
A_train[:,n_X+n_M:] = vector_preprocessing.fit_transform( v_train ).toarray()

In [ ]:
# Build preprocessing functions to select either only the word vectors or the other features
def select_word_vectors( A ):
    
    A_result = copy.copy( A )
    A_result[:,:n_X+n_M] = 0.
    
    return A_result

def select_semantic( A ):
    
    A_result = copy.copy( A )
    A_result[:,:n_X] = 0.
    A_result[:,n_X+n_M:] = 0.
    
    return A_result

def select_metadata( A ):
    
    A_result = copy.copy( A )
    A_result[:,n_X:] = 0.
    
    return A_result

In [ ]:
# Build the individual models.
vector_grad_boosting = Pipeline(
    [
        ( 'select_word_vectors', FunctionTransformer( select_word_vectors ) ),
        ( 'reg', GradientBoostingRegressor() ),
    ]
)

vector_knn = Pipeline(
    [
        ( 'select_word_vectors', FunctionTransformer( select_word_vectors ) ),
        ( 'reg', KNeighborsRegressor( n_neighbors=search.best_params_['knn__n_neighbors'] ) ),
    ]
)

semantic_linear_regression = Pipeline(
    [
        ( 'select_semantic', FunctionTransformer( select_semantic ) ),
        ( 'poly', PolynomialFeatures( degree=1 ) ),
        ( 'reg', LinearRegression( fit_intercept=False ) ),
    ]
)

metadata_grad_boosting = Pipeline(
    [
        ( 'select_metadata', FunctionTransformer( select_metadata ) ),
        ( 'reg', GradientBoostingRegressor() ),
    ]
)

neural_net = keras.models.Sequential()
neural_net.add( keras.layers.Input( shape=(n_X+n_M+n_v,) ) )
neural_net.add( keras.layers.Dense( 16, activation='relu', ) )
neural_net.add( keras.layers.Dense( 16, activation='relu' ) )
neural_net.add( keras.layers.Dense( 1, ) )
neural_net.compile( loss='mean_squared_error' )
vector_neural_net = Pipeline(
    [
        ( 'select_word_vectors', FunctionTransformer( select_word_vectors ) ),
        ( 'reg', neural_net ),
    ]
)

In [ ]:
model = VotingRegressor(
    estimators = [
        ( 'vector_grad_boosting', vector_grad_boosting ),
        ( 'vector_knn', vector_knn ),
        ( 'semantic_linear_regression', semantic_linear_regression ),
        ( 'metadata_grad_boosting', metadata_grad_boosting ),
        ( 'vector_neural_net', vector_neural_net ),
    ],
)

In [ ]:
# Perform KFold cross validation
# We could use the exact same code as before, but this allows a little additional control.
crossvals['voting'] = {'test_score':[]}
for train_index, test_index in tqdm.tqdm( kfold.split(A_train) ):
        
    # Fit and predict
    vector_grad_boosting.fit( A_train[train_index], y_train[train_index] )
    log_cpy_pred = vector_grad_boosting.predict( A_train[test_index] )
    
    # Compare prediction
    rmse = -np.sqrt( mean_squared_error( y_train[test_index], log_cpy_pred ) )
    crossvals['voting']['test_score'].append( rmse )
crossvals['voting']['test_score'] = np.array( crossvals['voting']['test_score'] )

In [ ]:
fig = rmse_swarmplot( crossvals, y_lim=(sample_std-0.5, sample_std+0.5 ) )

# Takeaways

* Even the best model can't predict citations that well. Our minimum RMSE suggests we will routinely misestimate by a factor of X

# Credits

Utilized python packages include:
* [ads](https://github.com/andycasey/ads)
* nltk